# ResNet18-Noah

In [2]:
import torch
!pip install torchsummary
from torchsummary import summary
from torch import nn
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


## Preparing train/test data and dataloader

In [6]:
train_transform = transforms.Compose([
    transforms.Resize(size=(128, 128)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.ToTensor()  
])

test_transform = transforms.Compose([
    transforms.Resize(size=(128,128)),
    transforms.ToTensor(),
    # normalize,
])


training_data = datasets.ImageFolder('/home/jovyan/aa_t6-adl-finalproj/frames/train', 
                                     transform = train_transform, 
                                     target_transform=None)

test_data = datasets.ImageFolder('/home/jovyan/aa_t6-adl-finalproj/frames/test', 
                                 transform = test_transform)

print(f"Train data:\n{training_data}\nTest data:\n{test_data}")


Train data:
Dataset ImageFolder
    Number of datapoints: 12376
    Root location: /home/jovyan/aa_t6-adl-finalproj/frames/train
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.2)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 102
    Root location: /home/jovyan/aa_t6-adl-finalproj/frames/test
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )


In [7]:
batch_size = 32

# Create data loaders.
'''train_dataloader = DataLoader(training_data, 
                              batch_size=batch_size)
test_dataloader = DataLoader(test_data, 
                             batch_size=batch_size)'''

                                                            # with new params
train_dataloader = DataLoader(training_data, 
                              batch_size=batch_size, 
                              shuffle=False, 
                              num_workers=2,
                              collate_fn=None,
                              pin_memory=True)

test_dataloader = DataLoader(test_data, 
                              batch_size=batch_size, 
                              shuffle=False, 
                              num_workers=2,
                              collate_fn=None,
                              pin_memory=True)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 3, 128, 128])
Shape of y: torch.Size([32]) torch.int64


## ResNet

In [8]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(
                in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(
                in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)

In [9]:
class ResBottleneckBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        self.downsample = downsample
        self.conv1 = nn.Conv2d(in_channels, out_channels//4,
                               kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(
            out_channels//4, out_channels//4, kernel_size=3, stride=2 if downsample else 1, padding=1)
        self.conv3 = nn.Conv2d(out_channels//4, out_channels, kernel_size=1, stride=1)

        if self.downsample or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=2 if self.downsample else 1),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()

        self.bn1 = nn.BatchNorm2d(out_channels//4)
        self.bn2 = nn.BatchNorm2d(out_channels//4)
        self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = nn.ReLU()(self.bn3(self.conv3(input)))
        input = input + shortcut
        return nn.ReLU()(input)

In [10]:
class ResNet(nn.Module):
    def __init__(self, in_channels, resblock, repeat, useBottleneck=False, outputs=1000):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        if useBottleneck:
            filters = [64, 256, 512, 1024, 2048]
        else:
            filters = [64, 64, 128, 256, 512]

        self.layer1 = nn.Sequential()
        self.layer1.add_module('conv2_1', resblock(filters[0], filters[1], downsample=False))
        for i in range(1, repeat[0]):
                self.layer1.add_module('conv2_%d'%(i+1,), resblock(filters[1], filters[1], downsample=False))

        self.layer2 = nn.Sequential()
        self.layer2.add_module('conv3_1', resblock(filters[1], filters[2], downsample=True))
        for i in range(1, repeat[1]):
                self.layer2.add_module('conv3_%d' % (
                    i+1,), resblock(filters[2], filters[2], downsample=False))

        self.layer3 = nn.Sequential()
        self.layer3.add_module('conv4_1', resblock(filters[2], filters[3], downsample=True))
        for i in range(1, repeat[2]):
            self.layer3.add_module('conv2_%d' % (
                i+1,), resblock(filters[3], filters[3], downsample=False))

        self.layer4 = nn.Sequential()
        self.layer4.add_module('conv5_1', resblock(filters[3], filters[4], downsample=True))
        for i in range(1, repeat[3]):
            self.layer4.add_module('conv3_%d'%(i+1,),resblock(filters[4], filters[4], downsample=False))

        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        self.fc = torch.nn.Linear(filters[4], outputs)
        
    def forward(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.layer4(input)
        input = self.gap(input)
        # torch.flatten()
        # https://stackoverflow.com/questions/60115633/pytorch-flatten-doesnt-maintain-batch-size
        input = torch.flatten(input, start_dim=1)
        input = self.fc(input)

        return input

In [11]:
resnet18 = ResNet(3, ResBlock, [2, 2, 2, 2], useBottleneck=False, outputs=2) 
resnet18.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(resnet18, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
         MaxPool2d-2           [-1, 64, 56, 56]               0
       BatchNorm2d-3           [-1, 64, 56, 56]             128
              ReLU-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,928
       BatchNorm2d-6           [-1, 64, 56, 56]             128
            Conv2d-7           [-1, 64, 56, 56]          36,928
       BatchNorm2d-8           [-1, 64, 56, 56]             128
          ResBlock-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,928
      BatchNorm2d-11           [-1, 64, 56, 56]             128
           Conv2d-12           [-1, 64, 56, 56]          36,928
      BatchNorm2d-13           [-1, 64, 56, 56]             128
         ResBlock-14           [-1, 64,

# Training the ResNet model

In [42]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(resnet18.parameters(), lr=2e-5, momentum=0.9)

# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(resnet18.parameters(), lr=2e-5)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=1e-4)

# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.RMSprop(resnet18.parameters(), lr=2e-5)

In [43]:
def train(dataloader, resnet18, loss_fn, optimizer):
    size = len(dataloader.dataset)
    resnet18.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = resnet18(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [44]:
def test(dataloader, resnet18, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    resnet18.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = resnet18(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [46]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, resnet18, loss_fn, optimizer)
    test(test_dataloader, resnet18, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.673778  [    0/12376]
loss: 0.000097  [ 3200/12376]
loss: 0.000046  [ 6400/12376]
loss: 0.000368  [ 9600/12376]
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.757022 

Epoch 2
-------------------------------
loss: 4.263470  [    0/12376]
loss: 0.000847  [ 3200/12376]
loss: 0.000323  [ 6400/12376]
loss: 0.000366  [ 9600/12376]
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.783515 

Epoch 3
-------------------------------
loss: 3.697777  [    0/12376]
loss: 0.007846  [ 3200/12376]
loss: 0.005739  [ 6400/12376]
loss: 0.002976  [ 9600/12376]
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.661655 

Epoch 4
-------------------------------
loss: 2.463339  [    0/12376]
loss: 0.022523  [ 3200/12376]
loss: 0.006960  [ 6400/12376]
loss: 0.006666  [ 9600/12376]
Test Error: 
 Accuracy: 50.0%, Avg loss: 1.314127 

Epoch 5
-------------------------------
loss: 2.649453  [    0/12376]
loss: 0.021482  [ 3200/12376]
loss: 0.006207  [ 6400/12376]
loss: 0.005211  [ 9600

In [52]:
torch.save(resnet18.state_dict(), "/home/jovyan/aa_t6-adl-finalproj/Outputs/model_resnet18_Adam1e-4.pth")
print("Saved PyTorch Model State to Outputs/model_resnet18_Adam1e-4.pth")

Saved PyTorch Model State to Outputs/model_resnet18_Adam1e-4.pth


## Loading model

In [ ]:
model = resnet18()
model.load_state_dict(torch.load("filepath/model_resnet18v2_Adam2e-5.pth"))
model.to(device)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def evaluate(model, test_dataloader):
    model.eval()
    true_labels = []
    predictions = []
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())
            predictions.extend(preds.cpu().numpy())
    cm = confusion_matrix(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    print("true_labels", true_labels, '\n')
    print("predictions", predictions, '\n')
    return cm, precision, recall, f1

cm, precision, recall, f1 = evaluate(resnet18, test_dataloader)
print('Confusion Matrix: \n', cm)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)